In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [2]:
conn = MongoClient(host='127.0.0.1', port=27017)  # 实例化MongoClient
db = conn.get_database('The_Wandering_Earth')  # 连接到数据库

# 猫眼数据以及保存为data.csv文件，无需从数据库中导出

douban = db.get_collection('douban') # 连接到集合comment
# maoyan = db.get_collection("maoyan")
douban_data = douban.find()
# maoyan_data = maoyan.find()

In [6]:
data1 = pd.json_normalize([comment for comment in douban_data])
# data2 = pd.json_normalize([comment for comment in maoyan_data])       #以数据库打开
data2 = pd.read_csv("data.csv")

In [7]:
data1.info()
print("\n")
data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102584 entries, 0 to 102583
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   content     102583 non-null  object 
 1   gender      102580 non-null  float64
 2   id          102580 non-null  float64
 3   nick        102579 non-null  object 
 4   replyCount  102580 non-null  float64
 5   score       102580 non-null  float64
 6   time        102580 non-null  object 
 7   upCount     102580 non-null  float64
 8   userId      102580 non-null  float64
 9   userLevel   102580 non-null  float64
dtypes: float64(7), object(3)
memory usage: 7.8+ MB


In [8]:
# 数据清洗
data1 = data1.drop(columns= "_id")
data1 = data1.drop(columns= "isWatched")
data1['time'] = pd.to_datetime(data1['time'], format='%Y-%m-%d')
data1 = data1[data1['time']>=pd.to_datetime('2019-02-01 00:00:00')]
data1.set_index(data1["time"], inplace=True)

print("\n")

data2 = data2.drop(columns='id')
data2 = data2.drop_duplicates(subset='userId')
data2['time'] = pd.to_datetime(data2['time'])

# 若使用数据库打开数据，数据时间字段的清洗用下面语句，将上一条语句注释
# data2['time'] = pd.to_datetime(data2['time']/1000, unit='s')

data2 = data2[data2['time']>=pd.to_datetime('2019-02-01 00:00:00')]
data2.set_index(data2["time"], inplace=True)
data1.info()
data2.info()

KeyError: "['_id'] not found in axis"

#### 一、数据详情
- 共有500条数据；
- 包含字段：
1.豆瓣：ID、评论内容、评论者昵称、是否观看过、评论时间、评分、评论点赞数、评论者所在城市
2.猫眼：评论内容、性别、评论ID、评论者昵称、回复数量、评分、时间、点赞数量、评论者ID、评论者等级
二、问题
1. 总体评价如何？
2. 总体评价的时间走向如何？
3. 高分的评价理由是什么？
4. 低分的评价理由是什么？
5. 低分的人群有哪些特征？（性别、等级）
6. 低分跟哪位演员有关？

 1. 总体评价如何？


In [ ]:
print(data1['rating'].mean())   # 3.382
print(data2["score"].mean())


score_total1 = data1['rating'].value_counts().sort_index()
score_total2 = data2['score'].value_counts().sort_index()
print(score_total1)
print(score_total2)


In [ ]:
from pyecharts.charts import Pie
from pyecharts.charts import Bar, Grid, Line
from pyecharts import options as opts
from pyecharts.components import Table


# 豆瓣：力荐=5，推荐=4，还行=3，较差=2，很差=1
# 猫眼：力荐=9-10，推荐=7-8，还行=5-6，较差=2-4，很差=1-0
rating_str = ["很差", "较差", "还行", "推荐", "力荐"]
rating_score1 = []
rating_score2 = [0,0,0,0,0]

# print(pd.value_counts(data1["rating"].values))

for i in range(1,6):
    rating_score1.append(pd.value_counts(data1["rating"].values)[i])

# print(pd.value_counts(data2["score"]))
for key,value in pd.value_counts(data2["score"]).items():
    # print(key,value)
    if key == 10 or key ==9:
        rating_score2[4] += value
    if key == 8 or key ==7:
        rating_score2[3] += value
    if key == 6 or key ==5:
        rating_score2[2] += value
    if key == 4 or key ==3:
        rating_score2[1] += value
    if key == 2 or key ==1:
        rating_score2[0] += value


#[['很差', 43], ['较差', 55], ['还行', 172], ['推荐', 128], ['力荐', 102]]
list_1 = [list(z) for z in zip(rating_str, rating_score1)]
list_2 = [list(z) for z in zip(rating_str, rating_score2)]
# print(list_2)
# print(list_1)

In [ ]:
# 展示豆瓣中评分饼图
c1 = (
    Pie()
    .add(
        "",
        [['很差', 43], ['较差', 55], ['还行', 172], ['推荐', 128], ['力荐', 102]],
        center=["30%", "45%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .add(
        "",
        list_2,
        center=["65%", "45%"],
        radius=[60, 80],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Rating_Pie"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    .render("./Charts_html/Rating_Pie.html")
)
print(1)

In [ ]:
list_ = [43, 55, 172, 128, 102]
# 展示豆瓣中评分折线图
bar = (
    Bar()
    .add_xaxis(rating_str)
    .add_yaxis("豆瓣", list_)
    .add_yaxis("猫眼", rating_score2)
    .set_global_opts(title_opts=opts.TitleOpts(title="评分分布柱状图"))
)
line = (
    Line()
    .add_xaxis(rating_str)
    .add_yaxis("豆瓣", list_)
    .add_yaxis("猫眼", rating_score2)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="评分分布折线图", pos_top="48%"),
        legend_opts=opts.LegendOpts(pos_top="48%"),
    )
)

grid = (
    Grid()
    .add(bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(line, grid_opts=opts.GridOpts(pos_top="60%"))
    .render("./Charts_html/Rating_line_bar.html")
)

print(1)

##### 2. 总体评价的时间走向如何？

In [ ]:

score_by_time1 = data1['rating'].resample('D').mean()
score_by_time2 = data2['score'].resample('D').mean()



douban_time = []
douban_score = []
for time,score in score_by_time1.items():
    douban_time.append(time)
    douban_score.append(score)


maoyan_time = []
maoyan_score = []
for time,score in score_by_time2.items():
    maoyan_time.append(time)
    maoyan_score.append(score)

# print(score_by_time1)
# print(score_by_time2)


c1 = (
    Bar()
    .add_xaxis(maoyan_time)
    .add_yaxis("猫眼评论", maoyan_score)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="猫眼评分随时间变化图"),
        datazoom_opts=[opts.DataZoomOpts()],
    )
    .render("./Charts_html/Score_by_time_maoyan.html")
)

c2 = (
    Bar()
    .add_xaxis(douban_time)
    .add_yaxis("豆瓣评论", douban_score)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="豆瓣评分随时间变化图"),
        datazoom_opts=[opts.DataZoomOpts()],
    )
    .render("./Charts_html/Score_by_time_douban.html")
)

print(1)




In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Map
from pyecharts.faker import Faker
import re

pattern = re.compile('[湖南|湖北|广东|广西|河南|河北|山东|山西|江苏|浙江|江西|黑龙江|'
                     '新疆|云南|贵州|福建|吉林|安徽四川|西藏|宁夏|辽宁|青海|'
                     '甘肃|陕西|内蒙古||台湾|北京|上海|天津]+')


info = data1['city'].value_counts()
province = info.keys()
amount = info.values

result = []

provinces = ['湖南', '湖北', '广东', '广西', '河南', '河北', '山东', '山西', '江苏', '浙江', '江西', '黑龙江', '新疆', '云南', '贵州', '福建', '吉林', '安徽', '四川', '西藏', '宁夏', '辽宁', '青海', '甘肃', '陕西', '内蒙古', '台湾', '北京', '上海', '天津']
temp_amount = []
temp_province = []


for i in range(len(province)):
    if pattern.findall(province[i]) != []:
        if pattern.findall(province[i])[0][:2] in temp_province:
            index = temp_province.index(pattern.findall(province[i])[0][:2])
            temp_amount[index] += int(amount[i])
        else:
            temp_province.append(pattern.findall(province[i])[0][:2])
            temp_amount.append(int(amount[i]))
        # result.append(temp)

result = [list(z) for z in zip(temp_province, temp_amount)]


c = (
    Map()
    .add("评论人数", result, "china")
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(max_=150, is_piecewise=True),
        title_opts=opts.TitleOpts(title="评论人数分布图"),
    )
    .render("./Charts_html/Map_amount.html")
)


In [ ]:
# 绘图需要导入的包

from pyecharts.charts import Scatter
from pyecharts.charts import Grid, Liquid, Page, Tab
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType

In [ ]:
rating = data1['rating'].values
likes = data1['likes'].values
likes_new = [0, 0, 0, 0, 0]
sum = 0


# 按评分统计点赞人数
for i in range(len(rating)):
    if int(rating[i]) == 1:
        likes_new[0] += int(likes[i])
    elif int(rating[i]) == 2:
        likes_new[1] += int(likes[i])
    elif int(rating[i]) == 3:
        likes_new[2] += int(likes[i])
    elif int(rating[i]) == 4:
        likes_new[3] += int(likes[i])
    else:
        likes_new[4] += int(likes[i])

for j in range(len(likes_new)):
    sum += likes_new[j]

favorate_rate = (likes_new[3] + likes_new[4]) / sum
poor_rate = (likes_new[0] + likes_new[1]) / sum
# print(type(favorate_rate), type(poor_rate))

# print(likes_new)
def scatter_rating_likes() -> Scatter:
    c = (
        Scatter()
        .add_xaxis(range(1,6))
        .add_yaxis("点赞数", likes_new)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="评分-点赞图"),
            visualmap_opts=opts.VisualMapOpts(type_="size", max_=100000, min_=20000),
        )
    )
    return c

def grid_liquid_favorate_rate() -> Grid:
    l1 = (
        Liquid().
        add(
        "好评率",
        [favorate_rate],
        center=["60%", "50%"],
        label_opts=opts.LabelOpts(
            font_size=50,
            formatter=JsCode(
                """function (param) {
                        return (Math.floor(param.value * 10000) / 100) + '%';
                    }"""
             ),
            position="inside",
            ),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="好评率和差评率"))
    )

    l2 = (
        Liquid().
        add(
        "差评率",
        [poor_rate],
        center=["25%", "50%"],
        label_opts=opts.LabelOpts(
            font_size=50,
            formatter=JsCode(
                """function (param) {
                        return (Math.floor(param.value * 10000) / 100) + '%';
                    }"""
             ),
            position="inside",
            ),
        )
    )

    grid = Grid().add(l1, grid_opts=opts.GridOpts()).add(l2, grid_opts=opts.GridOpts())
    return grid

# grid.render("./charts_html/scatter_rating_likes.html")
def page_layout():
    page = Page()
    page.add(
        scatter_rating_likes(),
        grid_liquid_favorate_rate()
    )
    page.render("./Charts_html/Rating_likes.html")

if __name__ == "__main__":
    page_layout()

### 文本情感分析：
##### 3. 高分的评价理由是什么？

In [ ]:
import jieba
from collections import Counter
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType

jieba.add_word('屈楚萧')
jieba.add_word('刘启')
jieba.add_word('吴京')
jieba.add_word('刘培强')
jieba.add_word('李光洁')
jieba.add_word('王磊')
jieba.add_word('吴孟达')
jieba.add_word('达叔')
jieba.add_word('韩子昂')
jieba.add_word('赵今麦')
jieba.add_word('韩朵朵')

swords = [x.strip() for x in open ('stopwords.txt', encoding="utf-8")]

print(swords)

In [ ]:
import re
pattern = re.compile('[^\d]+')

def plot_word_cloud_maoyan(data, swords, path):
    # print(data['content'])
    # 2019-02-11 10:10:00
    text = ''
    # print(type(data['content'].values[99]), data['content'].values[299])
    # 将所有评论连接起来

    for temp_i in data['content']:
        text += str(temp_i)

    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    # print(wc_data)

    c = (
    WordCloud()
    .add("", wc_data.values, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title=path))
    .render("./Charts_html/Maoyan_" + path + ".html")
    )


def plot_word_cloud_douban(data, swords, path):
    text = ''.join(data['comment'])
    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    print(wc_data)

    c = (
    WordCloud()
    .add("", wc_data.values, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title=path))
    .render("./Charts_html/Douban_" + path + ".html")
    )

In [ ]:
# 绘制豆瓣的评价的词云
# 高分
plot_word_cloud_douban(data=data1[data1['rating']>=4], swords=swords, path="Worldcloud_favorate")
# 低分
plot_word_cloud_douban(data=data1[data1['rating']<=2], swords=swords, path="Worldcloud_poor")

In [ ]:
# 豆瓣高分的部分评论
data1[data1['rating']>3]

In [ ]:
# 豆瓣低分的部分评论
data1[data1['rating']<3]

In [ ]:
# 豆瓣高分评论
# 数量过多，需要解除注释即可查看
# for i in data1[data1['rating']>3]['comment']:
#     print(i+'\n')

In [ ]:
# 豆瓣低分评论
# for i in data1[data1['rating']<3]['comment']:
#     print(i+'\n')

In [ ]:
# 绘制猫眼的评价的词云
# 高分
# import re
# pattern1 = re.compile('[^\d]+')

# 以下列第一种形式即可获得评论的字符串
# print(type(data2[data2['score']>7]['content'][3003]))
# print(pattern1.findall(data2[data2['score']>7]['content'][3003]))

plot_word_cloud_maoyan(data=data2[data2['score']>7], swords=swords, path="Worldcloud_favorate")
# 低分
plot_word_cloud_maoyan(data=data2[data2['score']<4], swords=swords, path="Worldcloud_poor")

In [ ]:
# 猫眼高分的部分评论
data2[data2['score']>7]


In [ ]:
# 猫眼低分的部分评论
data2[data2['score']<4]

In [ ]:
# # 猫眼高分评论
# 数量过多，需要解除注释即可查看
# for i in data2[data2['score']>7]['content']:
#     print(i+'\n')

In [ ]:
# # 猫眼低分评论
# for i in data2[data2['score']<4]['content']:
#     print(i+'\n')

#### 5. 评分的人群有哪些特征？（性别、等级）

##### 1). 评分和性别及等级的关系(都只有在猫眼中才有)

In [ ]:
# 评分和性别的关系
# 0代表未知，1代表男，2代表女

# 获得总体各个性别的个数
gender_total_maoyan = data2['gender'].value_counts().values
# print(gender_total_maoyan.values)

# 获得低分各个性别的个数
gender_low_maoyan = data2.loc[data2['score']<4, 'gender'].value_counts().values

# 获得低分各个性别的个数
gender_high_maoyan = data2.loc[data2['score']>7, 'gender'].value_counts().values

# print(data2['gender'].value_counts())

# 将对应性别个数转换为int类型
total_sex = []
for i in gender_total_maoyan:
    total_sex.append(int(i))

# 将低分对应性别个数转换为int类型
low_sex = []
for j in gender_low_maoyan:
    low_sex.append(int(j))
# 数据库中猫眼低分评论只有3条，2条没有说明性别，1条为男性,因此添加女性数为0
# low_sex.append(0)

# 将高分对应性别个数转换为int类型
high_sex = []
for k in gender_high_maoyan:
    high_sex.append(int(k))

sum1 = total_sex[1] + total_sex[2]
sum2 = high_sex[1] + high_sex[2]
sum3 = low_sex[1] + low_sex[2]

# 将未知人数去掉并且求得每一种男女所占比例
total_sex_per = [round(total_sex[1]/sum1 * 100, 2), round(total_sex[2]/sum1 * 100, 2)]
high_sex_per = [round(high_sex[1]/sum2 * 100, 2), round(high_sex[2]/sum2 * 100, 2)]
low_sex_per = [round(low_sex[1]/sum3 * 100, 2), round(low_sex[2]/sum3 * 100, 2)]
# low_per = [low_sex[1]/sum3, low_sex[2]/sum3]
# print(total_sex,high_per,low_sex)


In [ ]:
# 评分和等级的关系

# 获得总体各个等级的个数
level_total_maoyan = data2['userLevel'].value_counts().sort_index().values
# print(gender_total_maoyan.values)

# 获得低分各个等级的个数
level_low_maoyan = data2.loc[data2['score']<4, 'userLevel'].value_counts().sort_index().values

# 获得高分各个等级的个数
level_high_maoyan = data2.loc[data2['score']>7, 'userLevel'].value_counts().sort_index().values

# 用于查看用户等级和人数
# print(data2['userLevel'].value_counts().sort_index())
# print(data2.loc[data2['score']<4, 'userLevel'].value_counts().sort_index())
# print(data2.loc[data2['score']>7, 'userLevel'].value_counts().sort_index())
# print(level_high_maoyan,level_low_maoyan,level_total_maoyan)

# 将对应等级个数转换为int类型
total_level = []
for i in level_total_maoyan:
    total_level.append(int(i))

# 将低分对应等级个数转换为int类型
low_level = []
for j in level_low_maoyan:
    low_level.append(int(j))
# 数据库中猫眼低分评论没有1，3，4级的用户,因此令对应等级的人数为0


# 将高分对应等级个数转换为int类型
high_level = []
for k in level_high_maoyan:
    high_level.append(int(k))

sum1 = 0
sum2 = 0
sum3 = 0

# 计算总体，高分，低分不同等级的总人数
for i in range(len(total_level)):
    sum1 += total_level[i]
    sum2 += high_level[i]
    sum3 += low_level[i]

total_level_per = []
high_level_per = []
low_level_per = []

# 求各个等级的所占比例
for i in range(len(total_level)):

    total_level_per.append(round(total_level[i]/sum1 * 100, 2))
    high_level_per.append(round(high_level[i]/sum2 * 100, 2))
    low_level_per.append(round(low_level[i]/sum3 * 100, 2))

# print(total_level_per, high_level_per, low_level_per)


In [ ]:
# 绘制《流浪地球》观众性别，等级条形图和饼图

# 绘制性别分布条形图
def sex_distribution_bar() -> Bar:
    bar = (
        Bar()
        .add_xaxis(['未知', '男', '女'])
        .add_yaxis("总体性别分布", total_sex)
        .add_yaxis("高分性别分布", high_sex)
        .add_yaxis("低分性别分布", low_sex)
        .set_global_opts(title_opts=opts.TitleOpts(title="观众性别分布条形图"))
    )
    return bar

# 绘制性别分布饼图
def sex_distribution_pie() -> Pie:
    fn = """
        function(params) {
            if(params.name == '女')
                return '\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            return params.name + ' : ' + params.value + '%';
        }
        """

    def new_label_opts():
        return opts.LabelOpts(formatter=JsCode(fn), position="center")

    pie = (
        Pie()
        .add(
            "",
            [list(z) for z in zip(['男', '女'], total_sex_per)],
            center=["15%", "30%"],
            radius=[60, 80],
            label_opts=new_label_opts(),
        )
        .add(
            "",
            [list(z) for z in zip(['男', '女'], high_sex_per)],
            center=["40%", "30%"],
            radius=[60, 80],
            label_opts=new_label_opts(),
        )
        .add(
            "",
            [list(z) for z in zip(['男', '女'], low_sex_per)],        #没有女性特殊处理
            center=["65%", "30%"],
            radius=[60, 80],
            label_opts=new_label_opts(),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="观众性别分布饼图"),
            legend_opts=opts.LegendOpts(
                type_="scroll", pos_top="20%", pos_left="80%", orient="vertical"
            ),
        )
    )

    return pie

# 查看数据格式
# print([list(z) for z in zip(['男', '女'], low_sex_per)])
# print([list(z) for z in zip(['0', '1', '2', '3', '4', '5', '6'], total_level_per)])

# 绘制等级分布条形图
def level_distribution_bar() -> Bar:
    bar = (
        Bar()
        .add_xaxis([0, 1, 2, 3, 4, 5, 6])
        .add_yaxis("总体等级分布", total_level)
        .add_yaxis("高分等级分布", high_level)
        .add_yaxis("低分等级分布", low_level)
        .set_global_opts(title_opts=opts.TitleOpts(title="观众等级分布条形图"))
    )
    return bar

# 绘制等级分布饼图
def level_distribution_pie() -> Pie:
    fn = """
        function(params) {
            if(params.name == '5')
                return '\\n\\n\\n\\n\\n\\n\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '4')
                return '\\n\\n\\n\\n\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '3')
                return '\\n\\n\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '2')
                return '\\n\\n\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '1')
                return '\\n' + params.name + ' : ' + params.value + '%';
            if(params.name == '0' || params.name == '6')
                return ' ';

        }
        """

    def new_label_opts():
        return opts.LabelOpts(formatter=JsCode(fn), position="center")

    # 绘制饼图的时候1，2，3，4类型要是字符串类型
    pie = (
        Pie()
        .add(
            "",
            [list(z) for z in zip(['0', '1', '2', '3', '4', '5', '6'], total_level_per)],
            center=["15%", "30%"],
            radius=[80, 100],
            label_opts=new_label_opts(),
        )
        .add(
            "",
            [list(z) for z in zip(['0', '1', '2', '3', '4', '5', '6'], high_level_per)],
            center=["40%", "30%"],
            radius=[80, 100],
            label_opts=new_label_opts(),
        )
        .add(
            "",
            [list(z) for z in zip(['0', '1', '2', '3', '4', '5', '6'], low_level_per)],        #没有女性特殊处理
            center=["65%", "30%"],
            radius=[80, 100],
            label_opts=new_label_opts(),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="观众等级分布饼图"),
            legend_opts=opts.LegendOpts(
                type_="scroll", pos_top="20%", pos_left="80%", orient="vertical"
            ),
        )
    )

    return pie


# 将条形图和饼图放在一起展示
def page_layout1() -> Page:
    page = Page(layout=Page.SimplePageLayout)
    page.add(
        sex_distribution_bar(),
        sex_distribution_pie()

    )
    page.render("./Charts_html/Sex_distribution.html")
    return  page


def page_layout2() -> Page:
    page = Page(layout=Page.SimplePageLayout)
    page.add(
        level_distribution_bar(),
        level_distribution_pie()

    )
    page.render("./Charts_html/Level_distribution.html")
    return  page



if __name__ == "__main__":
    page_layout1()
    page_layout2()

# tab = Tab()
# tab.add(page_layout1(), "《流浪地球》观众性别分布")
# tab.add(page_layout2(), "《流浪地球》观众等级分布")
# tab.add(sex_distribution_bar(), "《流浪地球》观众性别分布条形图")
# tab.add(sex_distribution_pie(), "《流浪地球》观众性别分布饼图")
# tab.add(level_distribution_bar(), "《流浪地球》观众等级分布条形图")
# tab.add(level_distribution_pie(), "《流浪地球》观众等级分布饼图")

# tab.render("./charts_html/Sex_Level_distribution.html")

##### 6. 高低分跟哪位演员有关？

In [ ]:
# 将演员真名，真名绰号，角色绰号和电影角色名做一个映射
mapping = {'liucixin':'刘慈欣|大刘', 'guofan':'郭帆', 'quchuxiao':'屈楚萧|刘启|户口|男主', 'wujing':'吴京|刘培强|战狼',
           'liguangjie':'李光洁|王磊', 'wumengda':'吴孟达|达叔|韩子昂', 'zhaojinmai':'赵今麦|韩朵朵|女主'}

# 将演员名字的拼音作为数据的新字段，若该条记录出现了该演员，则将其名字的拼音对应为true
for key, value in mapping.items():
    data2[key] = data2['content'].str.contains(value)

In [ ]:
# 节约栈空间
# data2.info

### 总体提及次数

In [ ]:
# 将对应的人名拼音和在数据中的出现次数形成字典

# 因为数据存在Nah值，无法处理，因此pd.Series({key: data.loc[data[key], 'score']无法使用，但是使用分数≥0也可以做到相同效果
staff_count = pd.Series({key: data2.loc[data2[key]&(data2['score']>=0), 'score'].count() for key in mapping.keys()}).sort_index()
staff_count

In [ ]:
# 求低分评论中出现的演员名及其个数
staff_count_low = pd.Series({key: data2.loc[data2[key]&(data2['score']<4), 'score'].count() for key in mapping.keys()}).sort_index()
staff_count_low

In [ ]:
# 求高分评论中出现的演员名及其个数
staff_count_high = pd.Series({key: data2.loc[data2[key]&(data2['score']>7), 'score'].count() for key in mapping.keys()}).sort_index()
staff_count_high

In [ ]:
# 演员在低分评论的提及次数 占 所有评论中该演员的提及次数
staff_count_pct1 = np.round(staff_count_low/staff_count*100, 2).sort_index()
staff_count_pct1

In [ ]:
# 演员在高分评论的提及次数 占 所有评论中该演员的提及次数
staff_count_pct2 = np.round(staff_count_high/staff_count*100, 2).sort_index()
staff_count_pct2

In [ ]:
# 绘制演员在低分/高分评论的提及次数 占 所有评论中该演员的提及次数的柱状图

from pyecharts.globals import ThemeType
# 所有演员以下列顺序出现
# ['guofan', 'liguangjie', 'liucixin', 'quchuxiao', 'wujing', 'wumengda','zhaojinmai']

# 存放演员的总出现次数
count1 = []
# 存放好评中演员的总出现次数
count2 = []
# 存放差评中演员的总出现次数
count3 = []

for i in range(len(staff_count)):
    count1.append(int(staff_count[i]))
    count2.append(int(staff_count_high[i]))
    count3.append(int(staff_count_low[i]))

# print(count1,count2,count3)
list1 = []
list2 = []
list3 = []

# 将数据转换为指定的数据格式
'''
    [
    {"value": 12, "percent": 12 / (12 + 3)},
    {"value": 23, "percent": 23 / (23 + 21)},
    ...
    ]
'''

for i in range(len(staff_count)):
    temp1 = {}
    temp2 = {}
    temp3 = {}

    temp1["value"] = count2[i]
    temp1["percent"] = count2[i] / count1[i]
    list1.append(temp1)
    temp2["value"] = count3[i]
    temp2["percent"] = count3[i] / count1[i]
    list2.append(temp2)
    temp3["value"] = (count1[i] - count2[i] - count3[i])
    temp3["percent"] = (count1[i] - count2[i] - count3[i]) / count1[i]
    list3.append(temp3)


#### 低分评论所涉及到的主要演员名字

##### 1.吴孟达

In [ ]:
data2[data2['wumengda']&(data2['score']<4)].nlargest(5, 'upCount')

In [ ]:
for i in data2[data2['wumengda']&(data2['score']<4)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

##### 2.吴京

In [ ]:
data2[data2['wujing']&(data2['score']<4)].nlargest(5, 'upCount')

In [ ]:
for i in data2[data2['wujing']&(data2['score']<4)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

##### 3.赵金麦（女主）

In [ ]:
data2[data2['zhaojinmai']&(data2['score']<5)].nlargest(5, 'upCount')

In [ ]:
for i in data2[data2['zhaojinmai']&(data2['score']<5)].nlargest(5, 'upCount')['content']:
    print(i+'\n')


##### 4.屈楚萧（男主）

In [ ]:
data2[data2['quchuxiao']&(data2['score']<5)].nlargest(5, 'upCount')

In [ ]:
for i in data2[data2['quchuxiao']&(data2['score']<5)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

#### 高分评论所涉及到的主要演员名字

##### 1.吴孟达

In [ ]:
data2[data2['wumengda']&(data2['score']>7)].nlargest(5, 'upCount')

In [ ]:
for i in data2[data2['wumengda']&(data2['score']>7)].nlargest(5, 'upCount')['content']:
    print(i+'\n')


##### 2.吴京

In [ ]:
data2[data2['wujing']&(data2['score']>7)].nlargest(5, 'upCount')

In [ ]:
for i in data2[data2['wujing']&(data2['score']>7)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

##### 3.赵金麦（女主）

In [ ]:
data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount')

In [ ]:
for i in data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount')['content']:
    print(i+'\n')


##### 4.屈楚萧（男主）

In [ ]:
data2[data2['quchuxiao']&(data2['score']>7)].nlargest(5, 'upCount')

In [ ]:
for i in data2[data2['quchuxiao']&(data2['score']>7)].nlargest(5, 'upCount')['content']:
    print(i+'\n')

In [ ]:
from pyecharts.components import Table

# 高分评论
rows_h1 = []
rows_h2 = []
rows_h3 = []
rows_h4 = []
for i in range(5):
    comment1 = [data2[data2['wumengda']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    comment2 = [data2[data2['wujing']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    comment3 = [data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    comment4 = [data2[data2['quchuxiao']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    rows_h1.append(comment1)
    rows_h2.append(comment2)
    rows_h3.append(comment3)
    rows_h4.append(comment4)

# 低分评论
rows_l1 = []
rows_l2 = []
rows_l3 = []
rows_l4 = []
for i in range(5):
    comment1 = [data2[data2['wumengda']&(data2['score']<4)].nlargest(5, 'upCount').values[i][0]]
    comment2 = [data2[data2['wujing']&(data2['score']<4)].nlargest(5, 'upCount').values[i][0]]
    comment3 = [data2[data2['zhaojinmai']&(data2['score']<4)].nlargest(5, 'upCount').values[i][0]]
    comment4 = [data2[data2['quchuxiao']&(data2['score']<4)].nlargest(5, 'upCount').values[i][0]]
    rows_l1.append(comment1)
    rows_l2.append(comment2)
    rows_l3.append(comment3)
    rows_l4.append(comment4)

In [ ]:
c = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(['郭帆', '李光洁', '刘慈欣', '屈楚萧', '吴京', '吴孟达','赵今麦'])
    .add_yaxis("好评提及次数", list1, stack="stack1", category_gap="50%")
    .add_yaxis("差评提及次数", list2, stack="stack1", category_gap="50%")
    .add_yaxis("中评提及次数", list3, stack="stack1", category_gap="50%")
    .set_series_opts(
        label_opts=opts.LabelOpts(
            position="right",
            formatter=JsCode(
                "function(x){return Number(x.data.percent * 100).toFixed() + '%';}"
            ),
        )
    )
)
c.render("./Charts_html/Evaluate_bar.html")


In [ ]:
# 以下的表格画出来不好用
# 画好评差评中评在单条的条形图
def evaluate_bar() -> Bar:

    c = (
        Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
        .add_xaxis(['郭帆', '李光洁', '刘慈欣', '屈楚萧', '吴京', '吴孟达','赵今麦'])
        .add_yaxis("好评提及次数", list1, stack="stack1", category_gap="50%")
        .add_yaxis("差评提及次数", list2, stack="stack1", category_gap="50%")
        .add_yaxis("中评提及次数", list3, stack="stack1", category_gap="50%")
        .set_series_opts(
            label_opts=opts.LabelOpts(
                position="right",
                formatter=JsCode(
                    "function(x){return Number(x.data.percent * 100).toFixed() + '%';}"
                ),
            )
        )
    )
    return c


# 画高分评论表格
def table_baseh1() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h1

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="吴孟达")
    )
    return table

def table_baseh2() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h2

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="吴京")
    )
    return table

def table_baseh3() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h3

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="赵今麦")
    )
    return table

def table_baseh4() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h4

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="屈楚萧")
    )
    return table


# 画高分评论表格
def table_basel1() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h1

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="吴孟达")
    )
    return table

def table_basel2() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h2

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="吴京")
    )
    return table

def table_basel3() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h3

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="赵今麦")
    )
    return table

def table_basel4() -> Table:
    table = Table()

    headers = ["Content"]
    rows = rows_h4

    table.add(headers, rows).set_global_opts(
        title_opts=opts.ComponentTitleOpts(title="屈楚萧")
    )
    return table



def page_layout():
    page = Page(layout=Page.SimplePageLayout)
    page.add(
        evaluate_bar(),
        table_baseh1(),
        table_baseh2(),
        table_baseh3(),
        table_baseh4(),
        table_basel1(),
        table_basel2(),
        table_basel3(),
        table_basel4(),


    )
    page.render("evaluate_bar_table.html")


if __name__ == "__main__":
    page_layout()

In [ ]:
temp = data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount').values[0][0]
print(temp)
# for i in
comment = []
for i in range(5):
    temp= [data2[data2['zhaojinmai']&(data2['score']>7)].nlargest(5, 'upCount').values[i][0]]
    comment.append(temp)

print(comment)